In [2]:
from verification.utils import remove_softmax_activation

model_path = '../network/models/latest/model-verification'
remove_softmax_activation('../network/models/latest/model.h5', save_path=model_path)


INFO:tensorflow:Assets written to: ../network/models/latest/model-verification/assets


In [1]:
# import os, sys
# sys.path.remove(os.path.abspath('../verification'))
# sys.path.append(os.path.abspath('../.marabou'))
import pickle
import numpy as np
import pandas as pd
from robustness import LocalRobustness

model_path = '../network/models/latest/model.nnet'
out_dir = '../logs'
results_outpath = os.path.join(out_dir, 'results.csv')
counterexamples_outpath = os.path.join(out_dir, 'counterexamples.p')
n = 10
X, Y = pickle.load(open('../data/latest_new/verification.p', 'rb'))
X, Y = X[0:n], Y[0:n]

lr = LocalRobustness(
    network_path=model_path,
    network_options=dict(),
    marabou_options=dict(solveWithMILP=True, milpTightening='none'),
    X=X,
    Y=Y,
    e_min=0.0,
    e_max=1.0,
    e_interval=0.0001,
    timeout=0,
    verbosity=1
    )
lr.analyze(
    results_outpath=results_outpath,
    counterexamples_outpath=counterexamples_outpath
    )

# s0, class:4, pred:1, epsilon:0.02105712890625 (1472 ms)
# s1, class:0, pred:3, epsilon:0.0367431640625 (1234 ms)
# s2, class:4, pred:0, epsilon:0.00164794921875 (665 ms)
# s3, class:3, pred:4, epsilon:0.0108642578125 (1053 ms)
# s4, class:3, pred:1, epsilon:0.04534912109375 (710 ms)
# s5, class:2, pred:3, epsilon:0.02484130859375 (844 ms)
# s6, class:2, pred:3, epsilon:0.0162353515625 (713 ms)
# s7, class:0, pred:3, epsilon:0.04156494140625 (2885 ms)
# s8, class:2, pred:3, epsilon:0.0362548828125 (1548 ms)
# s9, class:0, pred:2, epsilon:0.01068115234375 (803 ms)

Instructions for updating:
non-resource variables are not supported in the long term
s0, class:4, pred:1, epsilon:0.02105712890625 (1474 ms)
s1, class:0, pred:3, epsilon:0.0367431640625 (1212 ms)
s2, class:4, pred:0, epsilon:0.00164794921875 (721 ms)
s3, class:3, pred:4, epsilon:0.0108642578125 (1172 ms)
s4, class:3, pred:1, epsilon:0.04534912109375 (788 ms)
s5, class:2, pred:3, epsilon:0.02484130859375 (923 ms)
s6, class:2, pred:3, epsilon:0.0162353515625 (762 ms)
s7, class:0, pred:3, epsilon:0.04156494140625 (3203 ms)
s8, class:2, pred:3, epsilon:0.0362548828125 (1679 ms)
s9, class:0, pred:2, epsilon:0.01068115234375 (853 ms)
completed analysis of 10 samples in 12 seconds


In [ ]:
lr.results

In [8]:
# import pickle
# import pandas as pd
# from verification.tot_net import TOTUtils

# feature_names = ('FixationDuration', 'FixationSeq', 'FixationStart', 'FixationX', 
#                  'FixationY', 'GazeDirectionLeftZ', 'GazeDirectionRightZ', 'PupilLeft', 
#                  'PupilRight', 'InterpolatedGazeX', 'InterpolatedGazeY', 'AutoThrottle', 
#                  'AutoWheel', 'CurrentThrottle', 'CurrentWheel', 'Distance3D', 'MPH', 
#                  'ManualBrake', 'ManualThrottle', 'ManualWheel', 'RangeW', 'RightLaneDist', 
#                  'RightLaneType', 'LeftLaneDist', 'LeftLaneType')
# label_names = ('fast', 'med_fast', 'med', 'med_slow', 'slow')

# train = pd.read_csv('../data/latest/train.csv', index_col=0)
# test = pd.read_csv('../data/latest/test.csv', index_col=0)

# output_cols = [f'TOT_{c}' for c in label_names]
# X_train, Y_train = train.drop(output_cols, axis=1).to_numpy(), train[output_cols].to_numpy()
# X_test, Y_test = test.drop(output_cols, axis=1).to_numpy(), test[output_cols].to_numpy()

# pickle.dump(X_train, open('../data/latest_new/X_train.p', 'wb'))
# pickle.dump(Y_train, open('../data/latest_new/Y_train.p', 'wb'))
# pickle.dump(X_test, open('../data/latest_new/X_test.p', 'wb'))
# pickle.dump(Y_test, open('../data/latest_new/Y_test.p', 'wb'))
# pickle.dump((X_train, Y_train), open('../data/latest_new/train.p', 'wb'))
# pickle.dump((X_test, Y_test), open('../data/latest_new/test.p', 'wb'))
# pickle.dump(feature_names, open('../data/latest_new/features.p', 'wb'))
# pickle.dump(label_names, open('../data/latest_new/labels.p', 'wb'))

# X_correct, Y_correct = TOTUtils.filter_incorrect_samples(model_path, X_test, Y_test, network_options=dict(modelType='savedModel_v2'))
# pickle.dump((X_correct, Y_correct), open('../data/latest_new/verification.p', 'wb'))